In [1]:
import numpy as np
import timeit 

In [37]:
def sortwithloops(input):
    L = input[:]
    ordered = False 
    while not ordered: 
        ordered = True
        for i in range(len(L)-1):
                if L[i] > L[i + 1]:
                   L[i] , L[i + 1] = L[i + 1] , L[i]
                   ordered = False
    return L 


def sortwithoutloops(input): 
    return sorted(input)


def sortwithoutloops_numpysort(input):
    return np.sort(input)
  

def searchwithloops(input, value):
    L = input[:]
    x = value
    for i in L:
         if i == x: 
            in_list = True 
            break
         else: 
            in_list = False
    return in_list #return a value


def searchwithoutloops(input, value):
    L = input[:]
    if value in L:
        return True
    else: 
        return False


def searchwithnumpy(input, value):
    x = np.where(np.asarray(input) == value)[0]
    if (x.size):
        return True
    else: 
        return False
   

 
if __name__ == "__main__":	
    L = [5,3,6,3,13,5,6]	
    
    print sortwithloops(L) # [3, 3, 5, 5, 6, 6, 13]  
    print sortwithoutloops(L) # [3, 3, 5, 5, 6, 6, 13] 
    print sortwithoutloops_numpysort(L)
    
    print searchwithloops(L, 5) #true  
    print searchwithloops(L, 11) #false
    print searchwithoutloops(L, 5) #true
    print searchwithoutloops(L, 11) #false
    
    print searchwithnumpy(L, 5)
    print searchwithnumpy(L, 11)
    
    import timeit
    
    print(timeit.timeit("sortwithloops(L)", setup="from __main__ import L, sortwithloops", number = 100000))
    print(timeit.timeit("sortwithoutloops(L)", setup="from __main__ import L, sortwithoutloops", number = 100000))
    print(timeit.timeit("sortwithoutloops_numpysort(L)", setup="from __main__ import L, sortwithoutloops_numpysort", number = 100000))
    

    print(timeit.timeit("searchwithloops(L, 5)", setup="from __main__ import L, searchwithloops", number = 100000))
    print(timeit.timeit("searchwithloops(L, 11)", setup="from __main__ import L, searchwithloops", number = 100000))
    print(timeit.timeit("searchwithoutloops(L, 5)", setup="from __main__ import L, searchwithoutloops", number = 100000))
    print(timeit.timeit("searchwithoutloops(L, 11)", setup="from __main__ import L, searchwithoutloops", number = 100000))
    
    print(timeit.timeit("searchwithnumpy(L, 5)", setup="from __main__ import L, searchwithnumpy", number = 100000))
    print(timeit.timeit("searchwithnumpy(L, 11)", setup="from __main__ import L, searchwithnumpy", number = 100000))
   
    

[3, 3, 5, 5, 6, 6, 13]
[3, 3, 5, 5, 6, 6, 13]
[ 3  3  5  5  6  6 13]
True
False
True
False
True
False
0.430781389304
0.0988478066245
0.391211102335
0.0437308167991
0.0781585246773
0.0363180943859
0.0511337420462
0.372249853859
0.415500142992
